# Data and package loads

In [1]:
"""
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install "tensorflow>=1.7"
!{sys.executable} -m pip install tensorflow-hub
!{sys.executable} -m pip install seaborn
"""

'\n# Install a pip package in the current Jupyter kernel\nimport sys\n!{sys.executable} -m pip install "tensorflow>=1.7"\n!{sys.executable} -m pip install tensorflow-hub\n!{sys.executable} -m pip install seaborn\n'

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import random

from IPython.display import display, Markdown


W0605 14:43:50.683374 140634140415744 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
imdb = pd.read_csv("imdb_master.csv", encoding="latin-1")
imdb.head(5)

text = imdb["review"][0:1000].tolist()

# print(text)

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(text))
    
"""
  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(text[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
"""

Instructions for updating:
Colocations handled automatically by placer.


W0605 14:43:54.620931 140634140415744 deprecation.py:323] From /home/chris/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


'\n  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):\n    print("Message: {}".format(text[i]))\n    print("Embedding size: {}".format(len(message_embedding)))\n    message_embedding_snippet = ", ".join(\n        (str(x) for x in message_embedding[:3]))\n    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))\n'

In [5]:
# this is an array of the embeddings, each is 512 long

text_array = np.array(message_embeddings).tolist()

# this is the text-  text

# dataframe

embed_frame = pd.DataFrame(
    {'text': text,
     'embedding': text_array
    })



In [6]:
def return_three_similar(value):
    to_sort = embed_frame["embedding"].apply(lambda i: np.inner(value, i))
    to_sort = (-to_sort).argsort()[1:3]
    #to_sort = to_sort.pop(0)
    return(to_sort)

test = list(return_three_similar(embed_frame["embedding"][0]))

embed_frame["similarity"] = embed_frame["embedding"].apply(lambda x: list(return_three_similar(x)))

# embed_frame.similarity

In [7]:
"""

I think I might just make one that works on the page, instead

f = open("test_similarity.html", "a")

pd.set_option("display.max_colwidth", 10000)

for counter, value in enumerate(embed_frame["text"]):
    f.write("<p>Review to be compared: ")
    f.write(value)
    f.write("</p>")
    f.write("<p>Most similar: ")
    f.write(embed_frame.text[embed_frame.similarity[counter][0]])#.to_string())
    f.write("</p>")
    f.write("<p>Second most similar: ")
    f.write(embed_frame.text[embed_frame.similarity[counter][1]])#.to_string())
    f.write("</p>")

f.close()

pd.set_option("display.max_colwidth", 50)

"""


'\n\nI think I might just make one that works on the page, instead\n\nf = open("test_similarity.html", "a")\n\npd.set_option("display.max_colwidth", 10000)\n\nfor counter, value in enumerate(embed_frame["text"]):\n    f.write("<p>Review to be compared: ")\n    f.write(value)\n    f.write("</p>")\n    f.write("<p>Most similar: ")\n    f.write(embed_frame.text[embed_frame.similarity[counter][0]])#.to_string())\n    f.write("</p>")\n    f.write("<p>Second most similar: ")\n    f.write(embed_frame.text[embed_frame.similarity[counter][1]])#.to_string())\n    f.write("</p>")\n\nf.close()\n\npd.set_option("display.max_colwidth", 50)\n\n'

In [8]:
row = random.randint(1, embed_frame.shape[0])

display(Markdown("<p>Review to be compared: "))
display(Markdown(embed_frame.text[row]))
display(Markdown("</p>"))
display(Markdown("<p>Most similar: "))
display(Markdown(embed_frame.text[embed_frame.similarity[row][0]]))
display(Markdown("</p>"))
display(Markdown("<p>Second most similar: "))
display(Markdown(embed_frame.text[embed_frame.similarity[row][1]]))
display(Markdown("</p>"))

<p>Review to be compared: 

First of all, I really can't understand how some people "enjoyed" this movie. It's the worst thing I have ever seen. Even the actors seem to be bored...and I think that says it all!<br /><br />However, I have to give my applause to the opening credits creators - that team seems to have a really good future. That's why I recommend the big studios to watch ONLY the opening credits, and one or two special effects sequences (if they're watched outside this movie, it almost looks like a good movie).<br /><br />Better luck (or judgment) next time for the producers of this, this... this "thing!".

</p>

<p>Most similar: 

Sandra Bullock is my favorite actress..... But this movie was so horrible, I couldn't help but chuckle throughout the movie in disbelief that I was actually watching something so crappy. Ha ha. The audio editing is horrible, They try too hard to come up with creative camera angles. Because they're just weird and stupid. The script sucked. Acting was horrible, storyline not very good. Very unrealistic, even for a movie. But it is a 20 year old movie..... so I'll give it a bonus point for that. And yeah, the music was terrible. But we all got to start somewhere. And submitting these things is such a hassle..... 10 line minimum... bother. Well now I know why I couldn't find this movie in the movie store.... I had to purchase it offline to see it... good thing it was only $.58 cents.... even though shipping was $2.59. Oh well.... I don't recommend anyone wasting their time and money seeing this film...

</p>

<p>Second most similar: 

This is one of those made-for-TV B movies that is so awful it kind of endears.<br /><br />Bad acting, predictable script and cheesy special effects that were pretty much some of the cheapest tat seen make you have to keep watching to see if it gets any better.<br /><br />It doesn't!

</p>